In [1]:
from functions import (create_shingles,
                        calculate_accuracy,
                        )

from preprocess_data_script import preprocess_data_script

import os
import json

import matplotlib.pyplot as plt
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, udf
from pyspark.sql.functions import broadcast

from pyspark.sql.functions import collect_list, collect_set, array
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import MinHashLSH, HashingTF

In [2]:
PROJECT_PATH = "/mnt/c/Users/lniedzwiedzki/Desktop/PDD2024/Proteins_LSH_Project"
SHINGLE = 5
BAND = 20
ROW = 5

In [3]:
spark = SparkSession.builder \
        .appName("LSH-Proteins") \
        .master("spark://spark-master:7077") \
        .getOrCreate()

print(f'The PySpark {spark.version} version is running...')

The PySpark 3.5.0 version is running...


In [4]:
os.chdir(PROJECT_PATH)

df, protein_df, data = preprocess_data_script(spark, PROJECT_PATH)

#df = spark.read.parquet(os.path.join(PROJECT_PATH, "data", "df.parquet"))
#protein_df = spark.read.parquet(os.path.join(PROJECT_PATH, "data", "proteins.parquet"))
#data = spark.read.parquet(os.path.join(PROJECT_PATH, "data", "data.parquet"))

(25112, 2)
(25112, 2)
(25111, 3)


In [5]:
df.show()

+---------------+----------+
|        cluster|      name|
+---------------+----------+
|UniRef50_Q8WZ42|    A2ASS6|
|UniRef50_Q8WZ42|  Q8WZ42-8|
|UniRef50_Q8WZ42|A0A2J8PRG4|
|UniRef50_Q8WZ42|A0A2J8PRH0|
|UniRef50_Q8WZ42|A0A2J8VRI6|
|UniRef50_Q8WZ42|A0A2J8VRF7|
|UniRef50_Q8WZ42|A0A8I5U7Y9|
|UniRef50_Q8WZ42|  Q8WZ42-2|
|UniRef50_Q8WZ42|A0A0C4DG59|
|UniRef50_Q8WZ42|  Q8WZ42-7|
|UniRef50_Q8WZ42|A0A2J8PRG6|
|UniRef50_Q8WZ42|A0A2J8VRH1|
|UniRef50_Q8WZ42|    C0JYZ2|
|UniRef50_Q8WZ42| Q8WZ42-11|
|UniRef50_Q8WZ42|    H2P803|
|UniRef50_Q8WZ42|A0A6P8QXT8|
|UniRef50_Q8WZ42|A0A6P8RJ11|
|UniRef50_Q8WZ42|A0A8B7X843|
|UniRef50_Q8WZ42|A0A091R2T7|
|UniRef50_Q8WZ42|A0A7L3SFD7|
+---------------+----------+
only showing top 20 rows



In [6]:
protein_df.show()

+----------+--------------------+
|      name|             protein|
+----------+--------------------+
|A0A014MIZ3|MSNDEKVLEYLKKLTAD...|
|A0A022PR68|MKDSITKKAKSILKDEL...|
|A0A026VSG9|MDLGGVRDITVRAGEDF...|
|A0A034WAF9|FIIRRKWQKTGNAIRAL...|
|A0A034WK66|VHEQVTPCEPNPCGSNA...|
|A0A034WPU1|APPNCRPECVVSSECSQ...|
|A0A034WTR2|VPNERAIFFVKIDCDED...|
|A0A034WV39|AHLACMNTRCADPCVGS...|
|A0A060W4G5|MWEAPEHDGGSPLTGYQ...|
|A0A060W4N1|MSSSSMTEMMSHSHLEG...|
|A0A060WEK7|MFIPESAVEIVRPPKDV...|
|A0A060YYB5|MRLIWKLPANDGGERIK...|
|A0A075R4X7|MNKDLIQLFSLTQPQQR...|
|A0A075THI1|SSITVAWGKPIYDGGSE...|
|A0A075TKX4|NSITVAWGKPIYDGGSE...|
|A0A076EI29|MGEVPLTIAQYIELRGD...|
|A0A087R0Q9|MVQHVHREFSPPSRLLR...|
|A0A087R0R0|VSGRPAPVITWSKQGVD...|
|A0A087R0R1|FRGKPVPNVTWNKSDTD...|
|A0A087R0R2|IKGRPEPEVKWEKAEGT...|
+----------+--------------------+
only showing top 20 rows



In [7]:
data.show()

+---------------+----------+--------------------+
|        cluster|      name|             protein|
+---------------+----------+--------------------+
|UniRef50_Q8WZ42|    A2ASS6|MTTQAPMFTQPLQSVVV...|
|UniRef50_Q8WZ42|  Q8WZ42-8|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A2J8PRG4|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A2J8PRH0|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A2J8VRI6|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A2J8VRF7|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A8I5U7Y9|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|  Q8WZ42-2|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A0C4DG59|AAEAVATGAKEVKQDAD...|
|UniRef50_Q8WZ42|  Q8WZ42-7|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A2J8PRG6|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A2J8VRH1|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|    C0JYZ2|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42| Q8WZ42-11|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|    H2P803|MTTQAPTFTQPLQSVVV...|
|UniRef50_Q8WZ42|A0A6P8QXT8|MTTQAPTFLQPLQSVVT...|
|UniRef50_Q8WZ42|A0A6P8RJ11|MTTQAPTFLQPLQSVVT...|


In [10]:
df = data.alias('df')
id(df) == id(data)  

False

In [14]:
numHashTables = int(BAND * ROW)

shingle_udf = udf(lambda x: create_shingles(x, k=SHINGLE), ArrayType(StringType()))

df = df.withColumn("shingles", shingle_udf(col("protein")))

hashingTF = HashingTF(inputCol="shingles", outputCol="features", numFeatures=1000)
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=numHashTables)

# Transform the data to include the features
featurizedData = hashingTF.transform(df)

model = mh.fit(featurizedData)

result = model.transform(featurizedData)

lsh_clusters = model.approxSimilarityJoin(result, result, 1.0, distCol="distance")

# Select only the necessary columns and filter self-joins
lsh_clusters = lsh_clusters.select(col("datasetA.cluster").alias("clusterA"),
                                   col("datasetB.cluster").alias("clusterB"),
                                   col("distance")).filter(col("clusterA") != col("clusterB"))

# Create a new DataFrame with unique cluster assignments
cluster_assignments = lsh_clusters.groupBy("clusterA").agg(collect_list("clusterB").alias("lsh_cluster"))

df_with_lsh = df.join(cluster_assignments, df.cluster == cluster_assignments.clusterA, how="left")

# Select the final columns
final_df = df_with_lsh.select("cluster", "name", "protein", "lsh_cluster")

final_df.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1231, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
                                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1067, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 529, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/bitnami/spark/python/lib/pyspark.zip/pyspark/worker.py", line 90, in read_command
    command = serializer._rea

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, adjusted_mutual_info_score

# Fill missing values in lsh_cluster with the original cluster values for comparison
df_with_lsh = df_with_lsh.withColumn("lsh_cluster", col("lsh_cluster").cast(StringType()))
df_with_lsh = df_with_lsh.withColumn("lsh_cluster", udf(lambda x: x if x else [], ArrayType(StringType()))(col("lsh_cluster")))
df_with_lsh = df_with_lsh.withColumn("lsh_cluster", udf(lambda x: x[0] if x else "N/A", StringType())(col("lsh_cluster")))

final_df = df_with_lsh.select("cluster", "lsh_cluster")
final_pd = final_df.toPandas()

true_labels = final_pd['cluster']
predicted_labels = final_pd['lsh_cluster']

accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)
fp = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
fn = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
adjusted_mi = adjusted_mutual_info_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"False Positives: {fp.sum()}")
print(f"False Negatives: {fn.sum()}")
print(f"Adjusted Mutual Information Score: {adjusted_mi}")

In [ ]:
spark.stop()